In [ ]:
import numpy as np
import struct 
import os
import plotly.graph_objects as go
import open3d as o3d

import chimera_fgo.util.general as general
from chimera_fgo.util.io import read_lidar_bin, read_gt
from chimera_fgo.registration import initialize_source_and_target, p2p_ICP, p2pl_ICP, p2pl_ICP_with_covariance
from chimera_fgo.util.geometry import se3_expmap, se3_logmap

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
np.set_printoptions(suppress=True, precision=3)

In [ ]:
binpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', '0027', 'velodyne_points', 'data')
PC_data = read_lidar_bin(binpath)

In [ ]:
ds_rate = 10
PC_data = [pc[::ds_rate] for pc in PC_data]

In [ ]:
%lprun -f read_lidar_bin read_lidar_bin(binpath)

In [ ]:
P1 = PC_data[0]
P2 = PC_data[1]

In [ ]:
fig = go.Figure(data=[general.pc_plot_trace(PC_data[100], size=1, color='blue', opacity=0.5)])
fig.update_layout(width=1600, height=900, 
    scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

In [ ]:
fig = go.Figure(data=[general.pc_plot_trace(P1, size=2), general.pc_plot_trace(P2, size=2, color='red')])
fig.update_layout(width=1600, height=900, 
    scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

## Adaptive Covariance Estimation (Shetty)

### LOAM feature extraction

In [ ]:
N = 100
P = np.random.rand(N, 3)

In [ ]:
# Covariance matrix
def pc_covariance(P):
    N = len(P)
    u = np.mean(P, axis=0)[None,:]
    C = (P.T @ P) / N - u.T @ u
    return C

In [ ]:
pc_covariance(P1)

In [ ]:
# Curvature
R = 0.3
i = 50
p = P[i]

# "Normal change rate" (from PCL)
# Local patch
local_idx = np.linalg.norm(P - p, axis=1) < R
p_local = P[local_idx]
C = pc_covariance(p_local)
eigvals, eigvecs = np.linalg.eig(C)
c = min(eigvals) / np.sum(eigvals)
c

In [ ]:
def get_curvatures(P, R=1.0):
    """Get curvatures of point cloud
    
    Parameters
    ----------
    P : np.array
        Point cloud
    
    Returns
    -------
    curvatures : np.array
        Curvatures of each point in point cloud

    """
    curvatures = np.zeros(P.shape[0])
    for i, p in enumerate(P):
        print(i)
        # Local patch
        local_idx = np.linalg.norm(P - p, axis=1) < R
        p_local = P[local_idx]
        C = pc_covariance(p_local)
        eigvals, eigvecs = np.linalg.eig(C)
        curvatures[i] = min(eigvals) / np.sum(eigvals)
    return curvatures

In [ ]:
curvatures = get_curvatures(P1)

In [ ]:
min(curvatures), max(curvatures)

In [ ]:
fig = go.Figure(data=general.pc_plot_trace(P1, size=2, color=curvatures))
fig.update_layout(width=1600, height=900, 
    scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

## Brossard ICP Covariance

In [ ]:
from scipy.spatial.transform import Rotation as R

r = R.from_euler('XYZ', [0, 0, 90], degrees=True)
R_rot = r.as_matrix()
R_rot

In [ ]:
P1 = (R_rot @ PC_data[0].T).T
P2 = (R_rot @ PC_data[1].T).T

fig = go.Figure(data=[general.pc_plot_trace(P1, size=2), general.pc_plot_trace(P2, size=2, color='red')])
fig.update_layout(width=1600, height=900, 
    scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

In [ ]:
T_ini = np.eye(4)
source, target = initialize_source_and_target(P1, P2)
#reg_p2l, Q_sensor, Q_init = p2pl_ICP_with_covariance(source, target, threshold=1, trans_init=T_ini, sigma=0.02, bias=0.05)
threshold=1
trans_init = T_ini
sigma=0.02
bias=0.05

In [ ]:
reg_p2pl = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())

# Sensor uncertainty term
T_rel = reg_p2pl.transformation 
corrs = np.asarray(reg_p2pl.correspondence_set)
target_normals = np.asarray(target.normals)
source_points = np.asarray(source.points)

B = np.zeros(6)
A = np.zeros((6,6))
for i,j in corrs:
    n = target_normals[j]
    p_T = (T_rel @ np.hstack((source_points[i], 1)))[:3]
    B_i = -np.hstack((np.cross(p_T, n), n))
    B += B_i
    A += B_i[:,None] @ B_i[:,None].T

A_inv = np.linalg.inv(A)
Q_sensor = sigma**2 * A_inv + A_inv @ B[:,None] * bias * B[:,None].T @ A_inv 

# Initialization noise term
T_ini = trans_init
Q_ini = 0.1*np.eye(6)  # TODO: set this
T_icp = T_rel

# Set sigma points
v_ini = 12 * [None]
v_ini[:6] = np.hsplit(np.sqrt(6 * Q_ini), 6)
v_ini[6:] = np.hsplit(-np.sqrt(6 * Q_ini), 6)

# Propagate sigma points through
T_ini_samples = 12 * [None]
T_icp_samples = 12 * [None]
v_icp = 12 * [None]

for i, v in enumerate(v_ini):
    T_ini_samples[i] = T_ini @ se3_expmap(v.flatten())
    T_icp_samples[i] = T_ini_samples[i] @ p2pl_ICP(source, target, threshold=1, trans_init=T_ini_samples[i])[0].transformation
    v_icp[i] = se3_logmap(np.linalg.inv(T_icp) @ T_icp_samples[i])

v_icp = np.array(v_icp)

# Compute covariance 
Q_init = (v_icp.T @ v_icp) / 12

In [ ]:
Q_sensor

In [ ]:
Q_init

In [ ]:
np.linalg.inv(Q_sensor+Q_init)

In [ ]:
0.2/np.sqrt(3)

In [ ]:
10/(180*np.sqrt(3))*np.pi

### Sensor noise term ($G Q_{sensor} G^T$)

In [ ]:
sigma = 0.02
bias = 0.05

T_ini = np.eye(4)
source, target = initialize_source_and_target(P1, P2)
reg_p2l = p2pl_ICP(source, target, threshold=1, trans_init=T_ini)[0]
T_rel = reg_p2l.transformation

In [ ]:
corrs = np.asarray(reg_p2l.correspondence_set)
target_normals = np.asarray(target.normals)

In [ ]:
np.asarray(source.points)

In [ ]:
B = np.zeros(6)
A = np.zeros((6,6))
for i,j in corrs:
    n = target_normals[j]
    p_T = (T_rel @ np.hstack((P1[i], 1)))[:3]
    B_i = -np.hstack((np.cross(p_T, n), n))
    B += B_i
    A += B_i[:,None] @ B_i[:,None].T

In [ ]:
A

In [ ]:
A_inv = np.linalg.inv(A)

In [ ]:
A_inv

In [ ]:
sigma**2 * A_inv + A_inv @ B[:,None] * bias * B[:,None].T @ A_inv 

### Initialization noise term ($(I - J) Q_{ini} (I - J)^T$)

In [ ]:
# Parameters
T_ini = np.eye(4)
Q_ini = np.eye(6)

source, target = initialize_source_and_target(P1, P2)
T_icp = p2pl_ICP(source, target, threshold=1, trans_init=T_ini)[0].transformation

In [ ]:
# Set sigma points
v_ini = 12 * [None]
v_ini[:6] = np.hsplit(np.sqrt(6 * Q_ini), 6)
v_ini[6:] = np.hsplit(-np.sqrt(6 * Q_ini), 6)

In [ ]:
# Propagate sigma points through
T_ini_samples = 12 * [None]
T_icp_samples = 12 * [None]
v_icp = 12 * [None]

for i, v in enumerate(v_ini):
    T_ini_samples[i] = T_ini @ se3_expmap(v.flatten())
    T_icp_samples[i] = T_ini_samples[i] @ p2pl_ICP(source, target, threshold=1, trans_init=T_ini_samples[i])[0].transformation
    v_icp[i] = se3_logmap(np.linalg.inv(T_icp) @ T_icp_samples[i])

v_icp = np.array(v_icp)

In [ ]:
# Compute covariance and infer J
cov = (v_icp.T @ v_icp) / 12
v_icp_mean = np.mean(v_icp, axis=0)
J = -(1/12) * (v_icp - v_icp_mean).T @ v_icp @ np.linalg.inv(Q_ini) + np.eye(6)

In [ ]:
cov

In [ ]:
np.linalg.eigvals(cov)

In [ ]:
np.linalg.inv(cov)

In [ ]:
J